# 模块D：监控与安全（Monitor & Security）测试

覆盖：stats/dashboard/history/diagnostics、配置管理（config as code/web）、告警（webhook/channels/anomaly）、限流/访问控制/审计、DDoS 基础防护、调度算法、TLS/插件。

提示：默认使用 `../build` 目录；如需重新编译，把 `BUILD_FIRST = True`。

In [ ]:
from __future__ import annotations

import os
import subprocess
from pathlib import Path

ROOT = (Path("..") if Path.cwd().name == "notebooks" else Path(".")).resolve()
BUILD_DIR = (ROOT / "build").resolve()

TIMEOUT_S = 30
BUILD_FIRST = False
CONTINUE_ON_FAIL = False

TESTS = [
    # monitoring surfaces
    "test_stats_json",
    "test_stats_backends",
    "test_history",
    "test_dashboard",
    "test_diagnostics",
    "test_config_as_code",
    "test_config_web",
    # alerts
    "test_alert_webhook",
    "test_alert_channels",
    "test_alert_anomaly",
    # security / protection
    "test_token_bucket",
    "test_per_ip_rate_limit",
    "test_per_path_rate_limit",
    "test_access_control",
    "test_audit_logger",
    "test_ddos_accept_limit",
    # traffic control / scheduling
    "test_congestion_control",
    "test_priority_queue",
    "test_fair_queue",
    "test_edf",
    # TLS + plugins (ops)
    "test_tls_acme",
    "test_plugin_manager",
    "test_plugin_http",
]


In [ ]:
def sh(cmd: list[str], *, cwd: Path = ROOT, timeout: int | None = None) -> None:
    print("$", " ".join(cmd))
    subprocess.run(cmd, cwd=str(cwd), check=True, timeout=timeout)


def maybe_build() -> None:
    BUILD_DIR.mkdir(parents=True, exist_ok=True)
    cache = BUILD_DIR / "CMakeCache.txt"
    if not cache.exists():
        sh(["cmake", "-S", str(ROOT), "-B", str(BUILD_DIR)])
    jobs = str(max(1, os.cpu_count() or 4))
    sh(["cmake", "--build", str(BUILD_DIR), "-j", jobs])


def run_tests() -> None:
    failures: list[tuple[str, str]] = []
    for t in TESTS:
        p = BUILD_DIR / t
        if not p.exists():
            print(f"SKIP {t} (missing: {p})")
            continue
        print(f"RUN {t}")
        try:
            sh([str(p)], cwd=ROOT, timeout=TIMEOUT_S)
        except subprocess.TimeoutExpired:
            print(f"FAIL {t} timeout={TIMEOUT_S}s")
            failures.append((t, "timeout"))
            if not CONTINUE_ON_FAIL:
                break
        except subprocess.CalledProcessError as e:
            print(f"FAIL {t} exit={e.returncode}")
            failures.append((t, f"exit={e.returncode}"))
            if not CONTINUE_ON_FAIL:
                break

    if failures:
        raise RuntimeError("Failed tests: " + ", ".join([f"{n}({r})" for n, r in failures]))
    print("OK")


if BUILD_FIRST:
    maybe_build()
run_tests()
